# Introduction

This notebook is for generating plots for Chapter 4.1 (Experimental Results, Synthetic Data - Binary).

It generates plots for decision boundaries of different linear binary classifiers for 2D data, learned from different loss functions.

For both a) separable and b) overlapping and both i) balanced and ii) imbalanced data.

# Imports and set-up

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from typing import List
import itertools

In [ ]:
import matplotlib.pylab as pylab

params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
pylab.rcParams.update(params)

Use GPU if available

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device")
print("-" * 60)
print(device)

# Linear classifier

A very simple linear neural network with input dimension 2 and output dimension 2 (the discriminant function for each class)

In [ ]:
class LinearClassifier(torch.nn.Module):
    def __init__(self, input_dim=2, output_dim=2):
        super(LinearClassifier, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.linear(x)
        return x

# Helper functions

Plotting function, extract weights and biases from model, training loop.

In [ ]:
def plot_db(wb: List[float], modelType: str = "[modelType]", lins="r"):
    """Plots decision boundary line where -4 <= x, y <= 4, given the weights
    and biases of the discriminant functions for each class. Options for
    plot title and linestyle.

    Args:
        wb (List[float]): List of weights and biases.
        modelType (str, optional): Label for plot. Defaults to '[modelType]'.
        lins (str, optional): Linestyle of plot. Defaults to 'r'.
    """

    w0, w1, b0, b1 = wb[0], wb[1], wb[2], wb[3]
    x0 = np.linspace(-4, 4, 100)
    x1 = (b1 - b0 - (w0[0] - w1[0]) * x0) / (w0[1] - w1[1])
    plt.plot(x0, x1, lins, label=f"{modelType}")

In [ ]:
def getwb(
    model: LinearClassifier, add: List[float] = [0, 0], mult: List[float] = [1, 1]
):
    """Takes in binary LinearClassifier that has been fitted. Returns the weights and
    biases of the two classes.

    Args:
        model (LinearClassifier): Linear Classifier
        add (list, optional): Optional post-hoc addition to model's biases. Defaults to [0, 0].
        mult (list, optional): Optional post-hoc scaling of each discriminant function. Defaults to [1, 1].

    Returns:
        List[float]: The weights and biases of the model, post any offsets and scaling.
    """
    _model = model.cpu()

    return [
        _model.linear.weight[0].detach().numpy() / mult[0],
        _model.linear.weight[1].detach().numpy() / mult[1],
        (_model.linear.bias[0].detach().numpy() + add[0]) / mult[0],
        (_model.linear.bias[1].detach().numpy() + add[1]) / mult[1],
    ]

In [ ]:
def modelTrain(
    model, optimizer, criterion, X_train, y_train, adjust: torch.Tensor = None
):
    """Trains a Pytorch model with an optimizer and criterion.
    When evaluating the loss, an adjustment can be applied to the output of the model.

    Args:
        model: Model to be trained.
        optimizer: Pytorch optimizer.
        criterion: Loss function
        X_train: Training X dataset.
        y_train: Training y dataset.
        adjust (torch.Tensor, optional): Additive update to output of model. Defaults to 0.
    """
    adjust = torch.Tensor([0]).to(device) if adjust is None else adjust
    for _ in range(10000):
        # zero the gradients
        optimizer.zero_grad()

        # forward
        output = model(X_train)
        loss = criterion(output + adjust, y_train.view(-1))

        # backward
        loss.backward()

        # optimize
        optimizer.step()

    return model

# Synthetic data generation and visualisation

10,000 samples

$ (100p)\% $ from Gaussian centered at 
$\begin{bmatrix}
-1 \\
-1
\end{bmatrix}$ 
with covariance $ \lambda I_2 $ where $ I_2 = \begin{bmatrix}
1 & 0\\
0 & 1
\end{bmatrix}$; 

the remaining $ (100(1-p))\% $ from Gaussian centered at $\begin{bmatrix}
1 \\
1
\end{bmatrix}$  with covariance $ \lambda I_2 $.

For a) separable, $ \lambda = 0.04 $, and for b) overlapping, $ \lambda = 1 $.

For i) balanced, $ p = 0.5 $, and for ii) imbalanced, $ p = 0.95 $.

In [ ]:
Xy_model_dict = {}

In [ ]:
N = 10000
for imbalanced, separable in list(itertools.product(*[[True, False]] * 2)):
    config = (imbalanced, separable)
    Xy_model_dict[config] = {}

    Xy_model_dict[config]["class_prob"] = [0.95, 0.05] if imbalanced else [0.5, 0.5]
    Xy_model_dict[config]["class_freq"] = [
        int(N * i) for i in Xy_model_dict[config]["class_prob"]
    ]
    λ = 0.04 if separable else 1

    covariance = λ * np.identity(2)
    np.random.seed(2022)  # fix seed for a reproducable training set
    Xy_model_dict[config]["sampA"] = np.random.multivariate_normal(
        [-1, -1], covariance, Xy_model_dict[config]["class_freq"][0]
    )
    Xy_model_dict[config]["sampB"] = np.random.multivariate_normal(
        [1, 1], covariance, Xy_model_dict[config]["class_freq"][1]
    )
    np.random.seed()

    X_values = np.concatenate(
        (Xy_model_dict[config]["sampA"], Xy_model_dict[config]["sampB"]), axis=0
    )  # N-by-2 data matrix
    y_flat = np.array(
        [0] * Xy_model_dict[config]["class_freq"][0]
        + [1] * Xy_model_dict[config]["class_freq"][1]
    )  # row vector of labels

    index_shuffle = np.arange(len(X_values))
    np.random.shuffle(index_shuffle)

    # shuffle the rows of data matrix and labels

    X_values = X_values[index_shuffle]
    y_flat = y_flat[index_shuffle]

    # convert to Torch tensors

    X_train = torch.tensor(X_values).type(torch.FloatTensor).to(device)
    y_train = torch.from_numpy(y_flat).view(-1, 1).type(torch.LongTensor).to(device)

    Xy_model_dict[config] |= {"X_train": X_train, "y_train": y_train}

In [ ]:
model_bin_orig = LinearClassifier().to(device)

# Training the classifiers

## Standard classifier

With single layer NN with Cross-Entropy Loss (for binary classification equivalent to a sigmoid classifier)

In [ ]:
for config in Xy_model_dict.keys():
    model_bin = deepcopy(model_bin_orig)
    optimizer_bin = torch.optim.SGD(model_bin.parameters(), lr=0.1, weight_decay=0)
    criterion_bin = torch.nn.CrossEntropyLoss().to(device)
    Xy_model_dict[config]["trained_model_bin"] = modelTrain(
        model_bin,
        optimizer_bin,
        criterion_bin,
        Xy_model_dict[config]["X_train"],
        Xy_model_dict[config]["y_train"],
    )

## Loss modified

Balance the loss

In [ ]:
for config in Xy_model_dict.keys():
    model_bin_bal = deepcopy(model_bin_orig)
    optimizer_bin_bal = torch.optim.SGD(
        model_bin_bal.parameters(), lr=0.1, weight_decay=0
    )
    criterion_bin_bal = torch.nn.CrossEntropyLoss(
        weight=torch.tensor(np.reciprocal(Xy_model_dict[config]["class_prob"])).type(
            torch.FloatTensor
        )
    ).to(device)
    Xy_model_dict[config]["trained_model_bin_bal"] = modelTrain(
        model_bin_bal,
        optimizer_bin_bal,
        criterion_bin_bal,
        Xy_model_dict[config]["X_train"],
        Xy_model_dict[config]["y_train"],
    )

In [ ]:
for config in Xy_model_dict.keys():
    beta = 0.999
    cui_weights = [
        (1 - beta) / (1 - beta**i) for i in Xy_model_dict[config]["class_freq"]
    ]  # inverse effective number for each class
    cui_weights = [i / sum(cui_weights) for i in cui_weights]  # normalise weights
    cui_weights = torch.tensor(cui_weights).type(torch.FloatTensor).to(device)

    model_bin_cui = deepcopy(model_bin_orig)
    optimizer_bin_cui = torch.optim.SGD(
        model_bin_cui.parameters(), lr=0.1, weight_decay=0
    )
    criterion_bin_cui = torch.nn.CrossEntropyLoss(weight=cui_weights).to(device)
    Xy_model_dict[config]["trained_model_bin_cui"] = modelTrain(
        model_bin_cui,
        optimizer_bin_cui,
        criterion_bin_cui,
        Xy_model_dict[config]["X_train"],
        Xy_model_dict[config]["y_train"],
    )

## Cao's LDAM loss

In [ ]:
# LDAM Loss taken from Cao et. al code and modified


class LDAMLoss(nn.Module):
    def __init__(self, class_prob, weight=None):
        super(LDAMLoss, self).__init__()
        delta = 1.0 / np.sqrt(np.sqrt(class_prob))
        delta = torch.FloatTensor(delta).to(device)
        self.delta = delta
        self.weight = weight

    def forward(self, outputs, target):
        # one-hot encodes the binary labels
        index = torch.zeros_like(outputs, dtype=torch.uint8).to(device)
        index.scatter_(1, target.data.view(-1, 1), 1)
        index_float = index.type(torch.FloatTensor).to(device)

        # column vector of the margin adjustments
        batch_m = torch.matmul(self.delta[None, :], index_float.transpose(0, 1))
        batch_m = batch_m.view((-1, 1))

        # computes logits modified by subtracting the margin for each example (determined by the true class) from the each output for it
        outputs_m = outputs - batch_m

        # for each example, replace the logit for the true class by the modified one, and keep the others the same
        output = torch.where(index, outputs_m, outputs)

        # computes the mean of cross-entropy losses for each example and its adjusted output
        return F.cross_entropy(output, target, weight=self.weight)

In [ ]:
for config in Xy_model_dict.keys():
    model_bin_cao = deepcopy(model_bin_orig)
    optimizer_bin_cao = torch.optim.SGD(
        model_bin_cao.parameters(), lr=0.1, weight_decay=0
    )
    criterion_bin_cao = LDAMLoss(Xy_model_dict[config]["class_prob"])
    Xy_model_dict[config]["trained_model_bin_cao"] = modelTrain(
        model_bin_cao,
        optimizer_bin_cao,
        criterion_bin_cao,
        Xy_model_dict[config]["X_train"],
        Xy_model_dict[config]["y_train"],
    )

## Equalised Loss

In [ ]:
# Loss with margin log(pi_j) for where j is the negative class


class EqLoss(nn.Module):
    def __init__(self, class_prob, weight=None):
        super(EqLoss, self).__init__()
        delta = np.log(class_prob)
        delta = torch.FloatTensor(delta).to(device)
        self.delta = delta
        self.weight = weight

    def forward(self, outputs, target):
        # one-hot encodes the binary labels
        index = torch.zeros_like(outputs, dtype=torch.uint8).to(device)
        index.scatter_(1, target.data.view(-1, 1), 1)

        # column vector of the margin adjustments
        batch_m = self.delta.repeat(outputs.shape[0], 1)

        # computes logits modified by adding the margin, log(pi_j), to each logit f_j for each example
        outputs_m = outputs + batch_m

        # for each example, replace the logit for the negative classes by the modified one in outputs_m, and keep original logit for true class
        output = torch.where(index, outputs, outputs_m)

        # computes the mean of cross-entropy losses for each example and its adjusted output
        return F.cross_entropy(output, target, weight=self.weight)

In [ ]:
for config in Xy_model_dict.keys():
    model_bin_eq = deepcopy(model_bin_orig)
    optimizer_bin_eq = torch.optim.SGD(
        model_bin_eq.parameters(), lr=0.1, weight_decay=0
    )
    criterion_bin_eq = EqLoss(Xy_model_dict[config]["class_prob"])
    Xy_model_dict[config]["trained_model_bin_eq"] = modelTrain(
        model_bin_eq,
        optimizer_bin_eq,
        criterion_bin_eq,
        Xy_model_dict[config]["X_train"],
        Xy_model_dict[config]["y_train"],
    )

## Logit-Adjusted Loss

In [ ]:
for config in Xy_model_dict.keys():
    la_margin_mat = (
        torch.Tensor(np.array([np.log(Xy_model_dict[config]["class_prob"])] * N))
        .type(torch.float32)
        .to(device)
    )

    model_bin_la = deepcopy(model_bin_orig)
    optimizer_bin_la = torch.optim.SGD(
        model_bin_la.parameters(), lr=0.1, weight_decay=0
    )
    criterion_bin_la = torch.nn.CrossEntropyLoss()
    Xy_model_dict[config]["trained_model_bin_la"] = modelTrain(
        model_bin_la,
        optimizer_bin_la,
        criterion_bin_la,
        Xy_model_dict[config]["X_train"],
        Xy_model_dict[config]["y_train"],
        adjust=la_margin_mat,
    )

# Plot the decision boundaries

Decision boundaries for each linear classifier (Vanilla ERM, logit-adjusted, reweighted and modified loss).

We also include the Bayes classifier decision boundaries (y = - x).

In [ ]:
for config in Xy_model_dict.keys():
    imbalanced, separable = config
    # Show the synthetic data
    plt.plot(
        Xy_model_dict[config]["sampA"][:, 0], Xy_model_dict[config]["sampA"][:, 1], "cx"
    )
    plt.plot(
        Xy_model_dict[config]["sampB"][:, 0], Xy_model_dict[config]["sampB"][:, 1], ".y"
    )
    # Plot the decision boundaries - can plot boundaries for different models
    # Plotting logit-adjusted models (multiplicative and additive updates) can be done as follows:

    # plot_db(getwb(model_bin, mult=class_prob), modelType='Multiplicative update', lins='g')
    # plot_db(getwb(model_bin, add=-np.log(class_prob)), modelType='Additive update', lins='b')

    plot_db(
        getwb(Xy_model_dict[config]["trained_model_bin"]),
        modelType="\u03B2 = 0",
        lins="r",
    )
    plot_db(
        getwb(Xy_model_dict[config]["trained_model_bin_cui"]),
        modelType=f"\u03B2 = {beta}",
        lins="b",
    )
    plot_db(
        getwb(Xy_model_dict[config]["trained_model_bin_bal"]),
        modelType="\u03B2 = 1",
        lins="g",
    )
    plot_db([[0, 0], [1, 1], 0, 0], modelType="Bayes classifier", lins="k--")
    # Add legend, title, axes labels, general formatting, saving the plot
    plt.legend(loc="best", fontsize=15)
    plt.title(
        f"Decision boundaries - {'Separable' if separable else 'Overlapping'}",
        fontsize=25,
    )
    plt.xlabel("x1", fontsize=20)
    plt.ylabel("x2", fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.grid()
    plt.axis("scaled")
    plt.axis([-4, 4, -4, 4])
    figure = plt.gcf()
    figure.set_size_inches(8, 8)
    plt.savefig(
        f"../data/{'imbal' if imbalanced else 'bal'}{'Sep' if separable else 'Overlap'}Weighted.png",
        dpi=1200,
    )
    plt.clf()

# Logit adjusted decision boundaries

In [ ]:
for config in Xy_model_dict.keys():
    imbalanced, separable = config

    plt.plot(
        Xy_model_dict[config]["sampA"][:, 0], Xy_model_dict[config]["sampA"][:, 1], "cx"
    )
    plt.plot(
        Xy_model_dict[config]["sampB"][:, 0], Xy_model_dict[config]["sampB"][:, 1], ".y"
    )
    # Plot decision boundaries for standard model with weights and biases adjusted according to update
    plot_db(
        getwb(
            Xy_model_dict[config]["trained_model_bin"],
            mult=Xy_model_dict[config]["class_prob"],
        ),
        modelType="Multiplicative update",
        lins="g",
    )
    plot_db(
        getwb(
            Xy_model_dict[config]["trained_model_bin"],
            add=-np.log(Xy_model_dict[config]["class_prob"]),
        ),
        modelType="Additive update",
        lins="b",
    )
    plot_db(
        getwb(Xy_model_dict[config]["trained_model_bin"]), modelType="ERM", lins="r"
    )
    plot_db([[0, 0], [1, 1], 0, 0], modelType="Bayes classifier", lins="k--")
    plt.legend(loc="best")
    plt.title(
        f"Decision boundaries - {'Separable' if separable else 'Overlapping'}",
        fontsize=25,
    )
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.grid()
    plt.axis("scaled")
    plt.axis([-4, 4, -4, 4])
    figure = plt.gcf()
    figure.set_size_inches(8, 8)
    plt.savefig(
        f"../data/{'imbal' if imbalanced else 'bal'}{'Sep' if separable else 'Overlap'}Update.png",
        dpi=1200,
    )
    plt.clf()

# Margin adjusted loss decision boundaries

In [ ]:
for config in Xy_model_dict.keys():
    imbalanced, separable = config

    plt.plot(
        Xy_model_dict[config]["sampA"][:, 0], Xy_model_dict[config]["sampA"][:, 1], "cx"
    )
    plt.plot(
        Xy_model_dict[config]["sampB"][:, 0], Xy_model_dict[config]["sampB"][:, 1], ".y"
    )
    plot_db(
        getwb(Xy_model_dict[config]["trained_model_bin"]), modelType="ERM", lins="r"
    )
    plot_db(
        getwb(Xy_model_dict[config]["trained_model_bin_cao"]),
        modelType="Cao/Adaptive",
        lins="g",
    )
    plot_db(
        getwb(Xy_model_dict[config]["trained_model_bin_eq"]),
        modelType="Tan/Equalised",
        lins="b",
    )
    plot_db(
        getwb(Xy_model_dict[config]["trained_model_bin_la"]),
        modelType="Logit Adjusted",
        lins="purple",
    )
    plot_db([[0, 0], [1, 1], 0, 0], modelType="Bayes classifier", lins="k--")
    plt.legend(loc="best")
    plt.title(
        f"Decision boundaries - {'Separable' if separable else 'Overlapping'}",
        fontsize=25,
    )
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.grid()
    plt.axis("scaled")
    plt.axis([-4, 4, -4, 4])
    figure = plt.gcf()
    figure.set_size_inches(8, 8)
    plt.savefig(
        f"../data/{'imbal' if imbalanced else 'bal'}{'Sep' if separable else 'Overlap'}Margin.png",
        dpi=1200,
    )
    plt.clf()